# Geração de embeddings: BAAI bge-3

Este notebook gera um arquivo .npy contendo embeddings - gerados a partir do modelo BAAI bge-3 - de um conjunto de textos. Para bom uso desse notebook, atente-se as informações sobre o modelo em questão.

Dados do modelo BAAI bge-3:

1-model_provider:

2-embed_arch:

3-embed_model:

4-max_sequence_length (Tokens):

5-n_parameters:

6-embed_dimension:

7-is_multilingual:

8- License:

9- source (HugginFace):

## Imports

In [20]:
import torch
import pandas as pd
import numpy as np
import os
import sys

from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

## Configurações

### Configura variáveis de execução

In [21]:
sep = ";"
dec = ","
quotech = "\""
encoding = "latin-1"


EP_dir = "EP2"
CSV_input_name = "ep2-train.csv"
path_to_archive = f"../../../Traindata/{EP_dir}/{CSV_input_name}"


do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Path to csv input is:  ../../../Traindata/EP2/ep2-train.csv


### Configura variáveis de reprodutibilidade

In [22]:
random_state = 12345

## Pré-tratamento dos dados

### Leitura dos dados

In [23]:
df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

(43678, 2)
Index(['req_text', 'profession'], dtype='object')


### Embaralhamento dos dados

O .csv de entrada tem alto ordenamento dos inputs por classe. Carregá-los dessa maneira nos modelos p/ treinamento introduz viés, então é preciso embaralhar os dados para garantir randomicidade. 
As classes em sklearn.model_selection - como a StratfiedKFold usada mais a frente - implementam parâmetro shuffle="", que pode ser passado como True para embaralhar mais os dados.

Note que é importante também garantir a reprodutibilidade do embaralhamento, especificando um valor hardcoded (Neste caso random_state=100)

In [24]:
print("Shape antes do shuffle:", df.shape)

df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

print("Shape depois do shuffle:", df.shape)

Shape antes do shuffle: (43678, 2)
Shape depois do shuffle: (43678, 2)


### Limpeza dos dados

In [25]:
# 1. Encontra o diretório atual do notebook
current_dir = os.getcwd()

# 2. Navega 3 níveis acima para encontrar a pasta raiz que contém 'Lib'
# Ajuste o número de 'os.pardir' (..) conforme a estrutura exata do seu diretório
# os.pardir representa ".."
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir, os.pardir))

# 3. Adiciona esse diretório ao caminho de busca do Python
sys.path.append(parent_dir)

In [26]:
from Lib.utils import printhello
from Lib.utils import clean_text

df['req_text_cleaned'] = df['req_text'].apply(lambda row_text: clean_text(
        row_text, 
        do_lowercase=True, 
        rem_emails=True, 
        rem_urls=True, 
        normalize_whitespaces=True
    ))

df['req_text'] = df['req_text_cleaned']
df = df.drop(columns=['req_text_cleaned']) # Remove a coluna temporária

print("Shape depois da limpeza:", df.shape)

Shape depois da limpeza: (43678, 2)


## Embeddings

### Função de geração de embeddings

In [27]:
def embbed_generator(df, model: str):
    embeddings_list = []
    device = "cuda" if torch.cuda.is_available() else "cpu"

    if model=="BAAI-bge-3":
        print("Gerando BAAI embeddings")
        
        embedding_model = SentenceTransformer('BAAI/bge-m3', device=device)
        
        embeddings_list = embedding_model.encode(df['req_text'].fillna("").tolist())
        print(f"Shape dos embeddings (X): {embeddings_list.shape}")
    elif model=="jiina-multilingual-v3":
        print("Gerando Jiina embeddings")
        embedding_model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True, device=device)
        #embedding_model = SentenceTransformer("jinaai/jina-embeddings-v3-base-multilingual", device=device)


        embeddings_list = embedding_model.encode(df['req_text'].fillna("").tolist())
    elif model=="gemma-300m":
        print("Gerando Gemma-300m embeddings")
        embedding_model = SentenceTransformer("google/embeddinggemma-300m", device=device)

        embeddings_list = embedding_model.encode_document(df['req_text'].fillna("").tolist())

    return embeddings_list

### Geração e salvamento de embeddings

In [28]:
embeddings_saving_path='./npys/embeddings_baai_bge3.npy'

In [29]:
# 1. Gerar os dados X e Y
X_baai = embbed_generator(df, "BAAI-bge-3")

print("embeddings gerados")

np.save(embeddings_saving_path, X_baai)
print(f"embeddings salvos em {embeddings_saving_path}")

Gerando BAAI embeddings
Shape dos embeddings (X): (43678, 1024)
embeddings gerados
embeddings salvos em ./npys/embeddings_baai_bge3.npy
